In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import praw
import datetime as dt
import random
import re
import string

import pickle
from compress_pickle import dump, load

import spacy
from spacy import displacy
from collections import Counter

import en_core_web_lg
nlp = en_core_web_lg.load()

STOP_WORDS = spacy.lang.en.stop_words.STOP_WORDS

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split


In [2]:
def contains_url_feature(comment):
    urlarr = []
    urlarr = re.findall('[(]?http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', str(comment))
    if not urlarr:
        return False
    else:
        return True

#comment_text is the the comment and art_doc is the cleaned text of the article
def wordscore_feature(comment_text, art_doc):
    art_doc = nlp(str(art_doc))
    art_items = [x.text for x in art_doc.ents]
    #get tokens
    art_tokens = []
    for (item, count) in Counter(art_items).most_common(5):
        token = nlp(item)[0]
        art_tokens += [token]
        
    doc = nlp(str(comment_text).lower())
    items = [x.text for x in doc.ents]
    
    score = 0
    
    for (item, count) in Counter(items).most_common(5):
        
        token = nlp(item)
        
        wordScores = []
        
        for art_word in art_tokens:
            
            wordScores += [art_word.similarity(token)]
            
            if len(wordScores) != 0:
                score += sum(wordScores)/len(wordScores)
            else:
                score = 0
    return score

#comment_text is the the comment and art_doc is the cleaned text of the article
def wholescore_feature(comment_text, art_doc):
    art_doc = nlp(str(art_doc))
    comment_text = str(comment_text).lower()
    doc = nlp(comment_text)
    score = art_doc.similarity(doc)
    return score

def remove_urls(text):
    urlarr = re.findall('[(]?http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', str(text))
    for url in urlarr:
        text = text.replace(url, '')
    return text

def remove_stopwords(text):
    text = nlp(str(text))
    token_list = []
    for token in text:
        token_list.append(token.text)
        
    filtered_text = ''
    
    for word in token_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            if word in string.punctuation:
                filtered_text += word
            else:
                filtered_text += (' ' + word)
    return filtered_text

def adjwordscore_feature(comment_text, art_doc):
    adjustments = {"PERSON":0.4, "NORP":0.25, "FAC": 0.35, "ORG": 0.075, "GPE": 0.2, "LOC": 0.15, "PRODUCT": 0, "EVENT": 0.2, "WORK_OF_ART": 0, "LAW": 0, "LANGUAGE": 0.2, "DATE": 0.4, "TIME": 0.6, "PERCENT": 0.8, "MONEY": 0.5, "QUANTITY": 0.5, "ORDINAL": 0.25, "CARDINAL": 0.2}
    art_doc = nlp(str(art_doc))
    art_items = [x.text for x in art_doc.ents]
    art_labels = [x.label_ for x in art_doc.ents]

    art_tokens = []
    for (item, count) in Counter(art_items).most_common(5):
        token = nlp(item)[0]
        art_tokens += [token]
    comment_text = str(comment_text)
    doc = nlp(str(comment_text).lower())

    items = [x.text for x in doc.ents]
    labels = [x.label_ for x in doc.ents]

    score = 0

    for (item, count) in Counter(items).most_common(5):
        com_label = labels[items.index(item)]
        token = nlp(item)

        wordScores = []

        for art_word in art_tokens:
            art_label = art_labels[art_tokens.index(art_word)]

            if art_label == com_label:
                amt = adjustments[art_label]
                wordScores += [art_word.similarity(token) - amt]
            else:
                wordScores += [art_word.similarity(token)]
        
        if len(wordScores) != 0:
            score += sum(wordScores)/len(wordScores)
        else:
            score = 0

    return score

def ner_feature(comment_text, art_doc):
    art_doc = nlp(str(art_doc))
    art_items = [x.text for x in art_doc.ents]
    art_tokens = []
    for item in Counter(art_items):
        token = nlp(item)[0]
        art_tokens += [token]
    comment_text = str(comment_text)
    doc = nlp(str(comment_text).lower())
    items = [x.text for x in doc.ents]

    score = 0

    for item in Counter(items):
        token = nlp(item)
        if str(token) in str(art_tokens):
            score += 1
    return len(items), score

def length_feature(comment):
    comment = str(comment)
    length = len(comment)
    return length

#Send in comment text, reddit url, and feature list, change to give features
def big_func(comment_text, article_text, features, model):
    feature_values = {'length': 0, 'WordScore': 0, 'WholeScore': 0, 'contains_url': False, 
                      'adjWordScore': 0, 'no_url_WordScore': 0, 'no_url_WholeScore': 0, 
                      'WordScoreNoStop': 0, 'WholeScoreNoStop': 0, 'no_url_or_stops_WholeScore': 0, 
                      'no_url_or_stops_WordScore': 0, 'NER_count': 0, 'NER_match': 0}
    if article_text == 'ERROR':
        return ['ERROR']
    
    feature_values['contains_url'] = contains_url_feature(comment_text)
    
    #Need to figure out how to do tfidf
    # feature_values['tfidf'] = tfidf_feature(comment_text, cleaned_article_text)

    feature_values['length'] = length_feature(comment_text)
    
    feature_values['WordScore'] = wordscore_feature(comment_text, article_text)
    feature_values['WholeScore'] = wholescore_feature(comment_text, article_text)
    
    feature_values['adjWordScore'] = adjwordscore_feature(comment_text, article_text)
    
    no_url_comment_text = remove_urls(comment_text)
    no_url_article_text = remove_urls(article_text)
    
    feature_values['no_url_WordScore'] = wordscore_feature(no_url_comment_text, no_url_article_text)
    feature_values['no_url_WholeScore'] = wholescore_feature(no_url_comment_text, no_url_article_text)
    
    comment_text = remove_stopwords(comment_text)
    article_text = remove_stopwords(article_text)
    
    feature_values['WordScoreNoStop'] = wordscore_feature(comment_text, article_text)
    feature_values['WholeScoreNoStop'] = wholescore_feature(comment_text, article_text)

    comment_text = remove_urls(comment_text)
    article_text = remove_urls(article_text)
    
    feature_values['no_url_or_stops_WholeScore'] = wholescore_feature(comment_text, article_text)
    feature_values['no_url_or_stops_WordScore'] = wordscore_feature(comment_text, article_text)
    
    feature_values['NER_count'], feature_values['NER_match'] = ner_feature(comment_text, article_text)
    
    prelim_features = []
    for feature in features:
        prelim_features.append(feature_values[feature])
    
    prelim_features = [prelim_features]
    prelim_features = np.array(prelim_features)
    
    prediction = model.predict(prelim_features)

    return prediction


In [3]:
comments = pd.read_csv('Validation Comments - Sheet1.csv')
comments

,Article Number,Comment text,related score,composure score,source score,gut feeling,total,Label,WordScore,WholeScore,...,no_url_WholeScore,WordScoreNoStop,WholeScoreNoStop,no_url_or_stops_WholeScore,no_url_or_stops_WordScore,NER_count,NER_match,length,profanity,model label
0,1,"What a farce. If not 19 then 20, if not 20 the...",7.0,3.0,0.0,3.0,13.0,0,6.989692,0.455088,...,0.455088,4.826901,0.457190,0.467852,4.826901,2,0,72,False,1
1,1,"""When the people find that they can vote thems...",7.0,5.0,3.0,3.0,18.0,0,0.000000,0.352507,...,0.352507,1.246337,0.339320,0.361756,1.246337,2,0,120,False,1
2,1,Interesting spin by the WSJ to note that the $...,8.0,4.0,4.0,5.0,21.0,1,2.059889,0.415445,...,0.415445,1.526402,0.379267,0.391046,1.526402,10,0,259,False,1
3,1,"""Mr. Biden said Saturday that the checks would...",3.0,2.0,3.0,0.0,8.0,0,0.290054,0.406714,...,0.406714,0.448589,0.421750,0.443767,0.448589,3,0,183,False,1
4,1,"Free money Free money, get your free money... ...",8.0,1.0,0.0,0.0,9.0,0,-0.240884,0.393341,...,0.393341,-0.240884,0.336735,0.354071,-0.240884,1,0,165,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,74,\nThe Democrats are no longer connected to the...,3.0,6.0,NaN,3.0,12.0,0,-1.080726,0.344537,...,0.344537,-0.649964,0.301120,0.322979,-0.649964,6,0,961,False,0
372,74,"The GOP acts like Biden owes them something, t...",3.0,6.0,NaN,3.0,12.0,0,-0.663746,0.359682,...,0.359682,-0.745070,0.328813,0.348438,-0.745070,8,0,515,False,0
373,74,The democrats left Joe Manchin years ago and I...,9.0,6.0,NaN,7.0,22.0,1,0.967275,0.366591,...,0.366591,0.967275,0.314389,0.327972,0.967275,3,0,111,False,1
374,74,Joe Biden said on the campaign trail that mine...,6.0,6.0,NaN,6.0,18.0,0,-0.427453,0.354744,...,0.354744,-0.427453,0.319018,0.335050,-0.427453,3,0,163,False,0


In [4]:
articles = pd.read_csv('Validation Articles - Sheet1.csv')
bad = articles[articles['text'] == 'Error']
bad

,Unnamed: 0,Article Number,Article URL/text,text


clf = load("../compressed_model.pkl", compression="lzma", set_default_extension=False)
features = ['WholeScore', 'contains_url','no_url_WholeScore',  'WholeScoreNoStop',
            'no_url_or_stops_WholeScore', 'NER_count', 'NER_match', 'length',
            'WordScore', 'adjWordScore', 'no_url_WordScore', 'WordScoreNoStop', 
            'no_url_or_stops_WordScore']
predictions = []
for index, row in comments.iterrows():
    comment_text = str(row['Comment text'])
    article_num = row['Article Number']
    article = articles[articles['Article Number'] == article_num]
    article_text = str(article['text'])
    predictions.append(big_func(comment_text, article_text, features, model))


In [5]:
features = ['WholeScore', 'contains_url','no_url_WholeScore',  'WholeScoreNoStop',
            'no_url_or_stops_WholeScore', 'NER_count', 'NER_match', 'length',
            'WordScore', 'adjWordScore', 'no_url_WordScore', 'WordScoreNoStop', 
            'no_url_or_stops_WordScore']
clf = load("../latest_model.pkl", compression="lzma", set_default_extension=False)
X = comments[features]
y = comments['Label']
predictions = []
for index, row in X.iterrows():
    feature_values = {'length': 0, 'WordScore': 0, 'WholeScore': 0, 'contains_url': False, 
                      'adjWordScore': 0, 'no_url_WordScore': 0, 'no_url_WholeScore': 0, 
                      'WordScoreNoStop': 0, 'WholeScoreNoStop': 0, 'no_url_or_stops_WholeScore': 0, 
                      'no_url_or_stops_WordScore': 0, 'NER_count': 0, 'NER_match': 0}
    prelim_features = []
    for feature in features:
        prelim_features.append(row[feature])
    
    prelim_features = [prelim_features]
    prelim_features = np.array(prelim_features)
    
    prediction = clf.predict(prelim_features)

    predictions.append(prediction)


/Users/Sam/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/Sam/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [6]:
comments['model label'] = predictions
comments

,Article Number,Comment text,related score,composure score,source score,gut feeling,total,Label,WordScore,WholeScore,...,no_url_WholeScore,WordScoreNoStop,WholeScoreNoStop,no_url_or_stops_WholeScore,no_url_or_stops_WordScore,NER_count,NER_match,length,profanity,model label
0,1,"What a farce. If not 19 then 20, if not 20 the...",7.0,3.0,0.0,3.0,13.0,0,6.989692,0.455088,...,0.455088,4.826901,0.457190,0.467852,4.826901,2,0,72,False,[True]
1,1,"""When the people find that they can vote thems...",7.0,5.0,3.0,3.0,18.0,0,0.000000,0.352507,...,0.352507,1.246337,0.339320,0.361756,1.246337,2,0,120,False,[False]
2,1,Interesting spin by the WSJ to note that the $...,8.0,4.0,4.0,5.0,21.0,1,2.059889,0.415445,...,0.415445,1.526402,0.379267,0.391046,1.526402,10,0,259,False,[False]
3,1,"""Mr. Biden said Saturday that the checks would...",3.0,2.0,3.0,0.0,8.0,0,0.290054,0.406714,...,0.406714,0.448589,0.421750,0.443767,0.448589,3,0,183,False,[False]
4,1,"Free money Free money, get your free money... ...",8.0,1.0,0.0,0.0,9.0,0,-0.240884,0.393341,...,0.393341,-0.240884,0.336735,0.354071,-0.240884,1,0,165,False,[False]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,74,\nThe Democrats are no longer connected to the...,3.0,6.0,NaN,3.0,12.0,0,-1.080726,0.344537,...,0.344537,-0.649964,0.301120,0.322979,-0.649964,6,0,961,False,[False]
372,74,"The GOP acts like Biden owes them something, t...",3.0,6.0,NaN,3.0,12.0,0,-0.663746,0.359682,...,0.359682,-0.745070,0.328813,0.348438,-0.745070,8,0,515,False,[False]
373,74,The democrats left Joe Manchin years ago and I...,9.0,6.0,NaN,7.0,22.0,1,0.967275,0.366591,...,0.366591,0.967275,0.314389,0.327972,0.967275,3,0,111,False,[False]
374,74,Joe Biden said on the campaign trail that mine...,6.0,6.0,NaN,6.0,18.0,0,-0.427453,0.354744,...,0.354744,-0.427453,0.319018,0.335050,-0.427453,3,0,163,False,[False]


In [7]:
comments.to_csv('Validation Comments - Sheet1.csv', index = False)

# Used file to featurize comments, stuff below is from analyzing previous model

In [8]:
#comments['Label'] = comments['Label'].replace({1: True, 0: False})
comments['model label'] = comments['model label'].replace({True: True, False: False})
comments

,Article Number,Comment text,related score,composure score,source score,gut feeling,total,Label,WordScore,WholeScore,...,no_url_WholeScore,WordScoreNoStop,WholeScoreNoStop,no_url_or_stops_WholeScore,no_url_or_stops_WordScore,NER_count,NER_match,length,profanity,model label
0,1,"What a farce. If not 19 then 20, if not 20 the...",7.0,3.0,0.0,3.0,13.0,0,6.989692,0.455088,...,0.455088,4.826901,0.457190,0.467852,4.826901,2,0,72,False,True
1,1,"""When the people find that they can vote thems...",7.0,5.0,3.0,3.0,18.0,0,0.000000,0.352507,...,0.352507,1.246337,0.339320,0.361756,1.246337,2,0,120,False,False
2,1,Interesting spin by the WSJ to note that the $...,8.0,4.0,4.0,5.0,21.0,1,2.059889,0.415445,...,0.415445,1.526402,0.379267,0.391046,1.526402,10,0,259,False,False
3,1,"""Mr. Biden said Saturday that the checks would...",3.0,2.0,3.0,0.0,8.0,0,0.290054,0.406714,...,0.406714,0.448589,0.421750,0.443767,0.448589,3,0,183,False,False
4,1,"Free money Free money, get your free money... ...",8.0,1.0,0.0,0.0,9.0,0,-0.240884,0.393341,...,0.393341,-0.240884,0.336735,0.354071,-0.240884,1,0,165,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,74,\nThe Democrats are no longer connected to the...,3.0,6.0,NaN,3.0,12.0,0,-1.080726,0.344537,...,0.344537,-0.649964,0.301120,0.322979,-0.649964,6,0,961,False,False
372,74,"The GOP acts like Biden owes them something, t...",3.0,6.0,NaN,3.0,12.0,0,-0.663746,0.359682,...,0.359682,-0.745070,0.328813,0.348438,-0.745070,8,0,515,False,False
373,74,The democrats left Joe Manchin years ago and I...,9.0,6.0,NaN,7.0,22.0,1,0.967275,0.366591,...,0.366591,0.967275,0.314389,0.327972,0.967275,3,0,111,False,False
374,74,Joe Biden said on the campaign trail that mine...,6.0,6.0,NaN,6.0,18.0,0,-0.427453,0.354744,...,0.354744,-0.427453,0.319018,0.335050,-0.427453,3,0,163,False,False


In [9]:
count = 0
for index, row in comments.iterrows():
    if row['Label'] == row['model label']:
        count += 1



In [10]:
count

220

In [11]:
count/376

0.5851063829787234

In [12]:
from sklearn.metrics import confusion_matrix

In [13]:
tn, fp, fn, tp = confusion_matrix(comments['Label'], comments['model label']).ravel()
print("True negative " + str(tn))
print("False Positive " + str(fp))
print("False negative " + str(fn))
print("True positive " + str(tp))

True negative 199
False Positive 22
False negative 134
True positive 21


In [14]:
confusion_matrix(comments['Label'], comments['model label'], normalize='true')

array([[0.90045249, 0.09954751],
       [0.86451613, 0.13548387]])

In [15]:
y = y.replace({"Good": True, "Bad":False})

TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'

In [ ]:
plot_confusion_matrix(clf, X, y, cmap='Blues', normalize = 'true', display_labels=['Good', 'Bad']);

In [ ]:
y_hat = clf.predict(X)
f1 = f1_score(y, y_hat)
acc = accuracy_score(y, y_hat)
print(f"Test f1 Score: {f1}")
print(f"Test acc Score: {acc}")

A lot of false positives, it looks like we were harsher than our model

In [ ]:
acc = 0
value = 0
for i in range(40):
    comments['Adjust Label'] = np.where(comments['total'] > i, 'Good', 'Bad')
    tn, fp, fn, tp = confusion_matrix(comments['Adjust Label'], comments['model label']).ravel()
    if acc <  (tn + tp)/376:
        acc = (tn + tp)/376
        value = i
comments['Adjust Label'] = np.where(comments['total'] > value, 'Good', 'Bad')
tn, fp, fn, tp = confusion_matrix(comments['Adjust Label'], comments['model label']).ravel()
print("True negative " + str(tn))
print("False Positive " + str(fp))
print("False negative " + str(fn))
print("True positive " + str(tp))
print("accuracy " + str(acc))
print("Good/bad cutoff " + str(value))

Most accurate when good is greater than 5 at 65%. But that just lowers our bar so far that all the good model predictions are right. Our model predicted more good comments than bad comments

In [ ]:
diff = 400
value = 0
for i in range(40):
    comments['Adjust Label'] = np.where(comments['total'] > i, 'Good', 'Bad')
    tn, fp, fn, tp = confusion_matrix(comments['Adjust Label'], comments['model label']).ravel()
    if diff >  abs(fp - fn):
        diff = abs(fp - fn)
        value = i

comments['Adjust Label'] = np.where(comments['total'] > value, 'Good', 'Bad')
tn, fp, fn, tp = confusion_matrix(comments['Adjust Label'], comments['model label']).ravel()

print("True negative " + str(tn))
print("False Positive " + str(fp))
print("False negative " + str(fn))
print("True positive " + str(tp))
print("difference " + str(diff))
print("accuracy " + str((tp + tn)/376))
print("Good/bad cutoff " + str(value))

Get the smallest difference between false positives and false negatives at cutoff of 14. Shows the cutoff that would be most balanced I think, and leads to a 60% accuracy.

In [ ]:
diff = 400
value = 0
for i in range(40):
    comments['Adjust Label'] = np.where(comments['total'] > i, 'Good', 'Bad')
    tn, fp, fn, tp = confusion_matrix(comments['Adjust Label'], comments['model label']).ravel()
    if diff >  abs(tp - tn):
        diff = abs(tp - tn)
        value = i

comments['Adjust Label'] = np.where(comments['total'] > value, 'Good', 'Bad')
tn, fp, fn, tp = confusion_matrix(comments['Adjust Label'], comments['model label']).ravel()

print("True negative " + str(tn))
print("False Positive " + str(fp))
print("False negative " + str(fn))
print("True positive " + str(tp))
print("difference " + str(diff))
print("accuracy " + str((tp + tn)/376))
print("Good/bad cutoff " + str(value))

In [ ]:
count = 0
for index, row in comments.iterrows():
    if row['Label'] == 'Good':
        count += 1
count